In [1]:
import os
from collections import OrderedDict

import torch
from src.util import (
    make_faces,
    vertex_tri_maps,
)

In [19]:
for i in [64, 128, 256, 512, 1000]:
    file = '/home/bobi/Desktop/pic2mesh/data/trimap_{}.pth'.format(i)
    print(file.split('/')[-1])
    loaded = torch.load(file)
    for key in loaded:
        print(key, loaded[key].shape)

trimap_64.pth
vert_tri_indices torch.Size([4096, 6])
vert_tri_weights torch.Size([1, 4096, 6, 1])
faces torch.Size([7938, 3])
trimap_128.pth
vert_tri_indices torch.Size([16384, 6])
vert_tri_weights torch.Size([1, 16384, 6, 1])
faces torch.Size([32258, 3])
trimap_256.pth
vert_tri_indices torch.Size([65536, 6])
vert_tri_weights torch.Size([1, 65536, 6, 1])
faces torch.Size([130050, 3])
trimap_512.pth
vert_tri_indices torch.Size([262144, 6])
vert_tri_weights torch.Size([1, 262144, 6, 1])
faces torch.Size([522242, 3])
trimap_1000.pth
vert_tri_indices torch.Size([1000000, 6])
vert_tri_weights torch.Size([1, 1000000, 6, 1])
faces torch.Size([1996002, 3])


In [20]:
64**2

4096

In [16]:
size = 64

faces = make_faces(size, size)
faces.shape

(7938, 3)

In [17]:
def vertex_tris(faces):
    res = [[] for _ in range(faces.max()+1)]
    for fid, face in enumerate(faces):        
        for vid in face:
            res[vid].append(fid)        
    return res

def vertex_tri_maps(faces):
    vts = vertex_tris(faces)
    r, c = len(vts), max([len(x) for  x in vts])
    vert_tri_indices = torch.zeros(r, c, dtype=torch.long)
    vert_tri_weights = torch.zeros(r, c)    
    for r, tris in enumerate(vts):        
        weight = 1. / len(tris)
        for c, tri_id in enumerate(tris):
            vert_tri_indices[r, c] = tri_id
            vert_tri_weights[r, c] = weight
    return vert_tri_indices, vert_tri_weights.unsqueeze(dim=-1)[None]


vert_tri_indices, vert_tri_weights = vertex_tri_maps(faces)
vert_tri_indices.shape, vert_tri_weights.shape

(torch.Size([4096, 6]), torch.Size([1, 4096, 6, 1]))

In [18]:
obj = OrderedDict(OrderedDict([
  ('vert_tri_indices', vert_tri_indices),
  ('vert_tri_weights', vert_tri_weights),
  ('faces', torch.tensor(faces)),
]))
file = './data/trimap_{}.pth'.format(size)
torch.save(obj, file)
file

'./data/trimap_64.pth'

In [11]:
size

128